# Demo IoT Environment Setup
![](../../Document/Images/Simulation.png)

## 1. Import Required Lib

In [ ]:
import requests
import json

## 2. Setup Env Var
**\_\_AIOTCORE\_\_ is the aiot-core publice interface's IP address of aiot-core container.**

**CB, CB_PORT, HM, HM_PORT, AIOT_CORE, AIOT_CORE_PORT and CRATEDB are defined in AIIOT-Project/Scripts/Setup/install.sh**

In [ ]:
__AIOTCORE__="http://127.0.0.1:9250"
CB="aiot-cb"
CB_PORT="1026"
HM="aiot-hm"
HM_PORT="8668"
AIOT_CORE="aiot-core"
AIOT_CORE_PORT="9250"
CRATEDB="aiot-cratedb"

data={
    "system_setting":{
        "ORION":f"http://{CB}:{CB_PORT}",
        "QUANTUMLEAP":f"http://{HM}:{HM_PORT}",
        "AIOTDFC":f"http://{AIOT_CORE}:{AIOT_CORE_PORT}",
        "CRATEDB":f"http://{CRATEDB}:4200"
    }}
r=requests.post(__AIOTCORE__+"/init",data=json.dumps(data))
print(r.status_code,r.text)

## 3. Setup Demo Environment Entitys

**3.1. Setup House Enitity**

In [ ]:
houseData={
   "type":"House",
   "id":"Rteslab",
   "metadata":
       {"timezone":"Asia/Taipei",
       "address":{
            "streetAddress":"No.1, University Road",
            "addressRegion":"Tainan City",
            "Country":"Taiwan,R.O.C."
            }
        } 
}
r=requests.post(__AIOTCORE__+"/entities",data=json.dumps(houseData))
print("House Set Status",r.status_code,r.text)

**3.2. Setup Floor Entities**

In [ ]:
floorDatas=[
    {
    "type":"Floor",
    "id":"1F",
    "metadata":{"relationship":"urn:ngsi-ld:House:Rteslab"}
    },
    {
    "type":"Floor",
    "id":"2F",
    "metadata":{"relationship":"urn:ngsi-ld:House:Rteslab"}
    },
    {
    "type":"Floor",
    "id":"B1",
    "metadata":{"relationship":"urn:ngsi-ld:House:Rteslab"}
    }
]
for floor in floorDatas:
    r=requests.post(__AIOTCORE__+"/entities",data=json.dumps(floor))
    print("Floor Set Status",r.status_code,r.text)

**3.3. Setup Room Entities**

In [ ]:
roomDatas=[
    {
    "type":"Room",
    "id":"Kitchen",
    "metadata":{"relationship":"urn:ngsi-ld:Floor:1F"}
    },
    {
    "type":"Room",
    "id":"Living Room",
    "metadata":{"relationship":"urn:ngsi-ld:Floor:2F"}
    },
    {
    "type":"Room",
    "id":"1F Rest Room",
    "metadata":{"relationship":"urn:ngsi-ld:Floor:1F"}
    },
    {
    "type":"Room",
    "id":"2F Rest Room",
    "metadata":{"relationship":"urn:ngsi-ld:Floor:2F"}
    },
    {
    "type":"Room",
    "id":"Basement",
    "metadata":{"relationship":"urn:ngsi-ld:Floor:B1"}
    }
]
for room in roomDatas:
    r=requests.post(__AIOTCORE__+"/entities",data=json.dumps(room))
    print("Room Set Status",r.status_code,r.text)

## 4. Setup Service Group

In [ ]:
service_group=[{"Name":"SG_Artifical"},{"Name":"SG_Real"},{"Name":"SG_Others"}]
r=requests.post(__AIOTCORE__+"/service-groups",data=json.dumps(service_group))
print(r.status_code,r.text)

## 5. Create IoT Devices

 **5.1. Setup IoT Agent IP address, which is tht domain name or ip of iotagent's container. Defined while install system, check the define in AIIOT-Project/Scripts/Setup/install.sh**
 

In [ ]:
iota_ul="http://aiot-iotagent:4041" 
# This is the combination of IOTA_UL:IOTA_UL_NPORT, 
# Which define in AIIOT-Project/Scripts/Setup/install.sh

iota_json="http://aiot-iotagent-json:4042"
# This is the combination of IOTA_JSON:IOTA_JSON_NPORT, 
# Which define in AIIOT-Project/Scripts/Setup/install.sh

**5.2. Install IoT Devices, where the fieldName is the dataset's name, which will need to match the picked dataset in [2.Pseudo_Data_Input](./2.Pseudo_Data_Input.ipynb), Step 4.**

In [ ]:
sensors=[{"agent_info":{
        "iotagent":iota_ul,
        "service_group":"SG_Artifical",
        },
        "sensor_info":{
            "deviceName":"Sensor01",
            "timeResolution":str(5*60),
            "targetTime":str(7*24*60*60),
            "targetModel":"HTM",
            "refRoom":"urn:ngsi-ld:Room:Kitchen",
            "savePeriod":"-1",
            "retrainPeriod":"0",
            "sensorName":"Artifically Daily Jumps Down",
            "fieldName":"art_daily_jumpsdown",
            "dataType":"Float",
            "sensorType":"Temp",
            "unit" : "Millisecond",
            "dummy": "True",
            "measurement": "COUNT"
            }
        }
    ,{"agent_info":{
        "iotagent":iota_json,
        "service_group":"SG_Artifical",
        },
        "sensor_info":{
            "deviceName":"Sensor02",
            "timeResolution":str(5*60),
            "targetTime":str(7*24*60*60),
            "targetModel":"HTM",
            "refRoom":"urn:ngsi-ld:Room:1F_Rest_Room",
            "savePeriod":"-1",
            "retrainPeriod":"0",
            "sensorName":"Artifically Daily Jumps Up",
            "fieldName":"art_daily_jumpsup",
            "dataType":"Float",
            "sensorType":"Temp",
            "unit" : "Millisecond",
            "dummy": "True",
            "measurement": "COUNT"
            }
        }
    ,{"agent_info":{
        "iotagent":iota_ul,
        "service_group":"SG_Real",
        },
        "sensor_info":{
            "deviceName":"Sensor01",
            "timeResolution":"300",
            "targetTime":str(7*24*60*60),
            "targetModel":"HTM",
            "refRoom":"urn:ngsi-ld:Room:Living_Room",
            "savePeriod":"-1",
            "sensorName":"Request Latency",
            "fieldName":"ec2_request_latency_system_failure",
            "retrainPeriod":"0",
            "dataType":"float",
            "sensorType":"Latency",
            "unit" : "Celsius",
            "dummy": "True",
            "measurement": "COUNT"}}
    ,{"agent_info":{
        "iotagent":iota_json,
        "service_group":"SG_Real",
        },
        "sensor_info":{
            "deviceName":"Sensor02",
            "timeResolution":"300", 
            "targetTime":str(7*24*60*60),
            "targetModel":"HTM",
            "refRoom":"urn:ngsi-ld:Room:2F_Rest_Room",
            "savePeriod":"-1",
            "sensorName":"Machine Temperature",
            "fieldName":"machine_temperature_system_failure",
            "retrainPeriod":"0",
            "dataType":"float",
            "sensorType":"Temperature",
            "unit" : "Celsius",
            "dummy": "True",
            "measurement": "COUNT"}}]
for d in sensors:
    r=requests.post(__AIOTCORE__+"/devices",data=json.dumps(d))
    print("Sensor Set Status",r.status_code,r.text)

## 6. Create Subscription

In [ ]:
service_group="SG_Real"
condition="AnomalyFlag"
expression={"Relational":"==","Value":"true"}
data={"Url":__AIOTCORE__+"/test","Condition":condition,"Expression":expression}
r=requests.post(__AIOTCORE__+"/subscriptions/"+service_group,data=json.dumps(data))
print(r.status_code,r.text)

## 7. Create Group Anomaly Detection Subscription

In [ ]:
service_group="SG_Artifical"
condition="GroupAnomaly"
policy="Default_Nearest_AND"
data={"Url":__AIOTCORE__+"/test","Condition":condition,"Policy":policy,"TimeWidth":86400}
r=requests.post(__AIOTCORE__+"/subscriptions/"+service_group,data=json.dumps(data))
print(r.status_code,r.text)

## 8. More Operation

## Reset All Setup

In [ ]:
r=requests.delete(__AIOTCORE__+"/reset")
print(r.status_code,r.text)

## Send Command

In [ ]:
data={
      'actionType':"modelControl",
      'action':"Train",
      'metadata':{"metadata":{"TrainLimit":30}}
     }
r=requests.post(__AIOTCORE__+"/devices/SG_Others/Sensor3/controls",data=json.dumps(data))
print(r.status_code,r.text)

## Remove Entity

In [ ]:
r=requests.delete(__AIOTCORE__+"/entities/urn:ngsi-ld:Floor:1F")
print(r.status_code,r.text)

## Remove Entity's Attributes

In [ ]:
r=requests.delete(__AIOTCORE__+"/entities/urn:ngsi-ld:House:Rteslab/attrs/address")
print(r.status_code,r.text)

## Remove Service Group

In [ ]:
r=requests.delete(__AIOTCORE__+"/service-groups/SG_Real")
print(r.status_code,r.text)

## Remove IoT Sensor

In [ ]:
r=requests.delete(__AIOTCORE__+"/devices/SG_Real/Sensor02")
print(r.status_code,r.text)

## Remove Subscription

In [ ]:
r=requests.delete(__AIOTCORE__+"/subscriptions/SG_Real/5ff36841453e43ed7861732d")
print(r.status_code,r.text)


## GET Method
* /entities -> List house information

* /entities/<entityID> -> Show <entitiyID>'s detail
* /attrs
* /attrs/<attributes>
* /attrs/<attributes>/value
* ?options=keyValues
* ?type=House,Floor,Room

* /devices/<service_group> -> List Device under <service_group>
* /devices/<service_group>/<deviceName> -> Show <deviceID>'s detail
* /devices/<service_group>/<deviceName>/model -> Show <deviceID>'s Model State
* /devices/<service_group>/<deviceName>/controls -> Show <deviceID>'s supported actionType and action
* /devices/<service_group>/<deviceName>/data -> List Sensor Data
* ?limit=<limit_count>
* ?attrs=a,b


* /subscriptions/<service_group> -> List Subscription under <service_group>
* /subscriptions/<service_group>/<subscriptionID> -> Show <subscriptionID>'s detail

* /service-groups -> List service-group
* /service-groups/<service_group> -> show <service_group>'s detail
* /service-groups/<service_group>/devices -> List Device under <service_group>
* /service-groups/<service_group>/subscriptions -> List Subscription under <service_group>

In [ ]:
r=requests.get(__AIOTCORE__+"/devices/SG_Artifical/")
print("URL : {url}\nStatus Code : {rtc}\nInfo : {info}".format(url=r.url,rtc=r.status_code,info=r.text))

## Update Entity's Attributes

In [ ]:
data={"actionType":"append","Name":{"type":"Text","value":"NCKU"}}
r=requests.patch(__AIOTCORE__+"/entities/urn:ngsi-ld:House:Rteslab",data=json.dumps(data))
print(r.status_code,r.text)